In [ ]:
以下代码解释：

1.读取CSV文件：使用pandas读取CSV文件并解析日期列。
2.计算每次间隙调整和速度调整的平均时间差：
• calculate_time_diff_threshold函数遍历所有间隙和速度列，计算每次间隙变化后的速度变化时间差，并筛选出时间差在10分钟以内的记录，最后返回平均时间差。
3.识别间隙和速度调整，并生成最终表格：
• analyze_adjustments函数遍历所有辊轮间隙列，检测每次间隙变化，查找每个间隙变化后的速度变化记录，筛选出在阈值时间内的记录，并将相关信息存储在结果列表中，最后转换为数据框。
4.结果输出：
• 最后调用analyze_adjustments函数进行分析，并输出结果数据框的前几行。

In [2]:
import pandas as pd
import numpy as np

# 读取CSV文件
combined_data = pd.read_csv("/Users/yue.a.tao/Desktop/Desktop/Gum/Data/combined_data_gap_speed.csv", parse_dates=['Date'])

# 计算每次间隙调整和速度调整的平均时间差
def calculate_time_diff_threshold(data, gap_cols, speed_cols, gap_threshold=0.001, speed_threshold=0.1):
    time_diffs = []

    for gap_col in gap_cols:
        # 计算间隙变化
        data[gap_col + '_change'] = data[gap_col].diff().fillna(0)
        gap_changes = data.loc[abs(data[gap_col + '_change']) > gap_threshold, ['Date']]

        for gap_time in gap_changes['Date']:
            for speed_col in speed_cols:
                # 计算速度变化
                data[speed_col + '_change'] = data[speed_col].diff().fillna(0)
                speed_changes = data.loc[abs(data[speed_col + '_change']) > speed_threshold, ['Date']]

                # 查找间隙变化后的第一个速度变化记录
                speed_change = speed_changes.loc[speed_changes['Date'] > gap_time, 'Date'].head(1)
                if not speed_change.empty:
                    time_diff = (speed_change.values[0] - gap_time) / np.timedelta64(1, 'm')
                    if time_diff <= 10:  # 筛选时间差在10分钟以内的记录
                        time_diffs.append(time_diff)

    # 返回平均时间差
    return np.mean(time_diffs)

# 主要函数：识别间隙和速度调整，并生成最终表格
def analyze_adjustments(data):
    gap_cols = ["1号辊间隙", "2号辊间隙", "3号辊间隙", "Forming Roller 定型辊间隙"]
    speed_cols = ["1号辊轮速度", "2号辊轮速度", "3号辊轮速度", "Forming Roller 辊轮速度"]

    # 计算所有辊轮间隙调整与速度调整之间的平均时间差
    threshold = calculate_time_diff_threshold(data, gap_cols, speed_cols)
    results = []

    for gap_col in gap_cols:
        # 检测间隙变化
        data[gap_col + '_change'] = data[gap_col].diff().fillna(0)
        gap_changes = data.loc[abs(data[gap_col + '_change']) > 0.001, ['Date', gap_col]]
        gap_changes['gap_before'] = gap_changes[gap_col].shift(1)
        gap_changes['gap_after'] = gap_changes[gap_col]

        for index, row in gap_changes.iterrows():
            gap_time = row['Date']

            for speed_col in speed_cols:
                # 检测速度变化
                data[speed_col + '_change'] = data[speed_col].diff().fillna(0)
                speed_changes = data.loc[abs(data[speed_col + '_change']) > 0.1, ['Date', speed_col]]
                speed_changes['speed_before'] = speed_changes[speed_col].shift(1)
                speed_changes['speed_after'] = speed_changes[speed_col]

                # 查找在阈值时间内的第一个速度变化记录
                speed_change = speed_changes.loc[(speed_changes['Date'] >= gap_time) & (speed_changes['Date'] <= (gap_time + pd.Timedelta(minutes=threshold))), :].head(1)

                if not speed_change.empty:
                    temp = {
                        'Item': data.loc[data['Date'] == gap_time, 'Item'].values[0],
                        'Sugar': data.loc[data['Date'] == gap_time, 'Sugar'].values[0],
                        'batch': data.loc[data['Date'] == gap_time, 'batch'].values[0],
                        'gap': gap_col,
                        'gap_change_time': gap_time,
                        'gap_before': row['gap_before'],
                        'gap_after': row['gap_after'],
                        'speed': speed_col,
                        'speed_change_time': speed_change['Date'].values[0],
                        'speed_before': speed_change['speed_before'].values[0],
                        'speed_after': speed_change['speed_after'].values[0]
                    }
                    results.append(temp)

    return pd.DataFrame(results)

# 调用函数进行分析
results = analyze_adjustments(combined_data)

# 打印结果
print(results.head())

           Item  Sugar  batch    gap     gap_change_time  gap_before  \
0   DMPE 绿箭原味薄荷  Sugar      2  1号辊间隙 2024-05-28 02:25:58         NaN   
1  DMRR（真叶玫瑰薄荷）  Sugar      6  1号辊间隙 2024-05-30 10:41:20      0.0997   
2  DMRR（真叶玫瑰薄荷）  Sugar      6  1号辊间隙 2024-05-30 10:41:20      0.0997   
3    DMRM（真叶薄荷）  Sugar      7  1号辊间隙 2024-05-30 23:14:58      0.1010   
4    DMRJ（茉莉薄荷）  Sugar      8  1号辊间隙 2024-05-31 04:10:05      0.1002   

   gap_after                speed   speed_change_time  speed_before  \
0     0.0997  Forming Roller 辊轮速度 2024-05-28 02:25:58         165.8   
1     0.1010               1号辊轮速度 2024-05-30 10:41:27          83.6   
2     0.1010  Forming Roller 辊轮速度 2024-05-30 10:42:05         165.6   
3     0.1002  Forming Roller 辊轮速度 2024-05-30 23:16:25         165.5   
4     0.1048               1号辊轮速度 2024-05-31 04:10:05          85.3   

   speed_after  
0        165.7  
1         83.7  
2        165.4  
3        165.4  
4          7.8  


In [3]:
# 保存结果为Excel文件
results.to_excel("/Users/yue.a.tao/Desktop/Desktop/Gum/Data/results_gap_speed.xlsx", index=False)

In [5]:
gap_cols = ["1号辊间隙", "2号辊间隙", "3号辊间隙", "Forming Roller 定型辊间隙"]
speed_cols = ["1号辊轮速度", "2号辊轮速度", "3号辊轮速度", "Forming Roller 辊轮速度"]


In [7]:
threshold = calculate_time_diff_threshold(combined_data, gap_cols, speed_cols)

In [8]:
print(threshold)

1.851113337149399
